In [6]:
import sys
sys.path.append('../')
import pandas as pd
from itertools import product
from game.Bankgames import GameTrueMatrix2by2
from game.distributions import TruncatedGaussian, PiecewiseUniform
from collections import Counter

In [7]:
def search_eps_signs():
    gamma_values = [0.4, 0.8]
#     gamma_values = [0.1, 0.2, 0.3, 0.7, 0.8, 0.9]
    mu_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    sigma_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    data = []
    
    # Iterate over all possible combinations
    for gamma_l, gamma_h, mu, sigma in product(gamma_values, gamma_values, mu_values, sigma_values):
        if gamma_l >= gamma_h:  # Ensure gamma_l < gamma_h
            continue
        
        taus = sorted([1 / (2 + gamma) for gamma in [gamma_l, gamma_h]])
        tg = TruncatedGaussian(mu=mu, sigma=sigma)
        gtm = GameTrueMatrix2by2(gammas=[gamma_l, gamma_h], taus=taus, dist=tg)
        
        sign_pair = gtm.eps_case  # Sign string either ++, -+, +- or --
        
        # Append data
        data.append((sign_pair, mu, sigma, gamma_l, gamma_h))
    
    # Create a DataFrame and sort by sign_pair
    df = pd.DataFrame(data, columns=["sign_pair", "mu", "sigma", "gamma_l", "gamma_h"])
    df_sorted = df.sort_values(by="sign_pair", ascending=True).reset_index(drop=True)
    
    return df_sorted

# Run the search and get the sorted DataFrame
df_result = search_eps_signs()
print(df_result)

   sign_pair   mu  sigma  gamma_l  gamma_h
0         ++  0.1    0.1      0.4      0.8
1         ++  0.3    0.1      0.4      0.8
2         ++  0.2    0.1      0.4      0.8
3         +-  0.4    0.1      0.4      0.8
4         +-  0.2    0.2      0.4      0.8
..       ...  ...    ...      ...      ...
76        --  0.3    0.9      0.4      0.8
77        --  0.3    0.8      0.4      0.8
78        --  0.3    0.7      0.4      0.8
79        --  0.4    0.6      0.4      0.8
80        --  0.9    0.9      0.4      0.8

[81 rows x 5 columns]


In [8]:
Counter(df_result['sign_pair'])

Counter({'--': 75, '++': 3, '+-': 3})

In [10]:
df_result[df_result['sign_pair'] == '+-'].to_csv("tg_pm_instances.csv", index=False, header=False)

In [19]:
df_result[['mu', 'sigma', 'gamma_l', 'gamma_h']].to_csv("tg_instances_slurm.csv", index=False, header=False)

In [20]:
df_result["sign_pair"].value_counts()

sign_pair
--    157
+-     47
++     36
Name: count, dtype: int64

In [15]:
import sys
sys.path.append('../')
import pandas as pd
from itertools import product
from game.Bankgames import GameTrueMatrix2by2
from game.distributions import TruncatedGaussian

def search_eps_signs():
    gamma_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    data = []
    for gamma_l, gamma_h in product(gamma_values, gamma_values):
        if gamma_l >= gamma_h:  # Ensure gamma_l < gamma_h
            continue
        
        taus = sorted([1 / (2 + gamma) for gamma in [gamma_l, gamma_h]])
        puf = PiecewiseUniform(ga_l = gamma_l, ga_h=gamma_h)
        gtm = GameTrueMatrix2by2(gammas=[gamma_l, gamma_h], taus=taus, dist=puf)
        
        sign_pair = gtm.eps_case  # Sign string either ++, -+, +- or --
        
        # Append data
        data.append((sign_pair, gamma_l, gamma_h))
    
    # Create a DataFrame and sort by sign_pair
    df = pd.DataFrame(data, columns=["sign_pair", "gamma_l", "gamma_h"])
    df_sorted = df.sort_values(by="sign_pair", ascending=True).reset_index(drop=True)
    
    return df_sorted

# Run the search and get the sorted DataFrame
df_result = search_eps_signs()
print(df_result)

   sign_pair  gamma_l  gamma_h
0         ++      0.1      0.2
1         ++      0.6      0.9
2         ++      0.6      0.8
3         ++      0.5      0.9
4         ++      0.5      0.8
5         ++      0.5      0.7
6         ++      0.4      0.9
7         ++      0.4      0.8
8         ++      0.4      0.7
9         ++      0.4      0.6
10        ++      0.3      0.9
11        ++      0.3      0.8
12        ++      0.3      0.7
13        ++      0.7      0.9
14        ++      0.3      0.5
15        ++      0.3      0.6
16        ++      0.2      0.9
17        ++      0.1      0.3
18        ++      0.1      0.4
19        ++      0.1      0.5
20        ++      0.1      0.6
21        ++      0.1      0.7
22        ++      0.1      0.9
23        ++      0.1      0.8
24        ++      0.2      0.4
25        ++      0.2      0.5
26        ++      0.2      0.6
27        ++      0.2      0.7
28        ++      0.2      0.8
29        ++      0.2      0.3
30        -+      0.3      0.4
31      

In [16]:
df_result["sign_pair"].value_counts()

sign_pair
++    30
-+     5
--     1
Name: count, dtype: int64

In [17]:
df_result[['gamma_l', 'gamma_h']].to_csv("puf_instances_slurm.csv", index=False, header=False)